In [1]:
! pip install bitsandbytes transformers peft accelerate evaluate --quiet
! pip install datasets trl ninja packaging --quiet
! pip install flash-attn --no-build-isolation --progress-bar off --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 51.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are 

In [3]:
import torch
import numpy as np
import pandas as pd
import os
import sys
import json
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)
from trl import SFTTrainer

In [6]:
from huggingface_hub import login
login()

In [7]:
# base model
model_name = "meta-llama/Llama-2-7b-hf"
# device
device = 'cuda'
# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [8]:
#Quantization as defined https://huggingface.co/docs/optimum/concept_guides/quantization will help us reduce the size of the model for it to fit on a single GPU
#Quantization configuration
compute_dtype = getattr(torch, "float16")
print(compute_dtype)
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)

torch.float16


In [9]:
# load and quantize model
model = AutoModelForCausalLM.from_pretrained(
          model_name,
          quantization_config=bnb_config,
          attn_implementation="flash_attention_2",
          device_map={"": 0},   #device_map="auto" caused a problem in the training

)

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [11]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): L

In [10]:
#Quantization as defined https://huggingface.co/papers/2309.15223 will help us reduces the number of parameters that need to be fine-tuned and it helps tremendously on performance on a single GPU
# LoRA configuration
peft_config = LoraConfig(
        lora_alpha=8,
        lora_dropout=0.01,
        r=16,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj", "lm_head",]
)

In [12]:
model = prepare_model_for_kbit_training(model)
# Configure the pad token in the model
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False # Gradient checkpointing is used by default but not compatible with caching

In [14]:
training_arguments = TrainingArguments(
        output_dir="./results",   # directory in which the checkpoint will be saved.
        eval_strategy="steps",    # options - epoch, steps, no. default 'no'
        optim="paged_adamw_8bit", # optimizer used with QLoRA
        per_device_train_batch_size=8, # train batch size
        per_device_eval_batch_size=8, # evaluation batch size
        gradient_accumulation_steps=2, #number of lines to accumulate gradient. cuation: it changes the size of a "step".
        log_level="debug", # options -  ‘info’, ‘warning’, ‘error’ and ‘critical’
        save_steps=150, # number of steps between checkpoints
        logging_steps=50, # number of steps between logging of the loss for monitoring
        learning_rate=5e-5,
        fp16=True, # mix precision floating point
        num_train_epochs=3,
        weight_decay=0.01,
        warmup_steps=50,
        lr_scheduler_type="constant",
        report_to="tensorboard"
)

In [15]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [16]:
train_data_file_path = "/content/gdrive/MyDrive/Colab Notebooks/thesis/data/northwind/window_4/train.csv"
val_data_file_path = "/content/gdrive/MyDrive/Colab Notebooks/thesis/data/northwind/window_4/val.csv"

In [17]:
# Load the dataset
dataset = load_dataset('csv', data_files={'train': [train_data_file_path], 'validation': [val_data_file_path]}, column_names=['prompt', 'response'])

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [18]:
test_file_path = "/content/gdrive/MyDrive/Colab Notebooks/thesis/data/northwind/window_4/test/products.csv"

In [19]:
test_dataset = load_dataset('csv', data_files={'test': [test_file_path]}, column_names=['prompt', 'response'])

Generating test split: 0 examples [00:00, ? examples/s]

In [20]:
def formatting_test_prompts_func(example):
  output_texts = []

  for i in range(len(example['prompt'])):
      if example['prompt'][i] == 'prompt':
        pass
      else:
        text = f"### Input: ```{example['prompt'][i]}```\n ### Output: ''"
        output_texts.append(text)

  return output_texts

In [21]:
eval_prompt = formatting_test_prompts_func(test_dataset['test'])

In [22]:
base_model_outputs = []
for test_data in eval_prompt:
  model_input = tokenizer(test_data, return_tensors="pt").to("cuda")
  with torch.no_grad():
    output = tokenizer.decode(model.generate(**model_input, max_length=512,tokenizer=tokenizer, stop_strings='</root>', pad_token_id=2)[0], skip_special_tokens=True)
    print(output)
    base_model_outputs.append(output)

The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.float16.


### Input: ```5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1
6,Grandma's Boysenberry Spread,3,2,12 - 8 oz jars,25.0,120,0,25,0
7,Uncle Bob's Organic Dried Pears,3,7,12 - 1 lb pkgs.,30.0,15,0,10,0
8,Northwoods Cranberry Sauce,3,2,12 - 12 oz jars,40.0,6,0,0,0
```
 ### Output: ''

# [154. Find the Duplicate Number](https://leetcode.com/problems/find-the-duplicate-number/)

## Description

Given an array of integers that might contain duplicates, return **any** duplicate in the array.

Provided is a list of numbers, and you need to return any number that appears more than once.

For example, if the list is `[1, 2, 3, 1, 3, 4, 2, 5, 2, 10, 9, 10, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 6
### Input: ```36,Inlagd Sill,17,8,24 - 250 g  jars,19.0,112,0,20,0
37,Gravad lax,17,8,12 - 500 g pkgs.,26.0,11,50,25,0
38,Côte de Blaye

In [ ]:
pd.DataFrame(base_model_outputs).to_csv("/content/gdrive/MyDrive/Colab Notebooks/thesis/data/northwind/window_4/test/output/northwind_window4_llama2_base.csv")

In [23]:
def formatting_prompts_func(example):
  output_texts = []

  for i in range(len(example['prompt'])):

      text = f"### Input: ```{example['prompt'][i]}```\n ### Output: {example['response'][i]}"

      output_texts.append(text)

  return output_texts

In [24]:
  trainer = SFTTrainer(
        model=model,
        train_dataset=dataset['train'],
        eval_dataset=dataset['validation'],
        peft_config=peft_config,
        formatting_func=formatting_prompts_func,
        #packing = True
        max_seq_length=512,
        tokenizer=tokenizer,
        args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/2415 [00:00<?, ? examples/s]

Map:   0%|          | 0/613 [00:00<?, ? examples/s]

Using auto half precision backend


In [25]:
trainer.train()

Currently training with a batch size of: 8
***** Running training *****
  Num examples = 2,415
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 453
  Number of trainable parameters = 40,554,496
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
50,0.742100,0.483320
100,0.433100,0.363876
150,0.344000,0.298648
200,0.268200,0.252999
250,0.229200,0.226045
300,0.220600,0.201973
350,0.183600,0.188184
400,0.181700,0.178874
450,0.171700,0.174069


***** Running Evaluation *****
  Num examples = 613
  Batch size = 8
***** Running Evaluation *****
  Num examples = 613
  Batch size = 8
***** Running Evaluation *****
  Num examples = 613
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-150
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:180: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
tokenizer config file saved in ./results/checkpoint-150/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-150/special_tokens_map.json
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve

TrainOutput(global_step=453, training_loss=0.30728946839999943, metrics={'train_runtime': 2186.7447, 'train_samples_per_second': 3.313, 'train_steps_per_second': 0.207, 'total_flos': 1.447727131310162e+17, 'train_loss': 0.30728946839999943, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 613
  Batch size = 8


{'eval_loss': 0.1796512007713318,
 'eval_runtime': 35.6356,
 'eval_samples_per_second': 17.202,
 'eval_steps_per_second': 2.161,
 'epoch': 3.0}

In [ ]:
!cp -r ./results/runs /content/gdrive/MyDrive/tensorboard

In [ ]:
model_outputs = []
for test_data in eval_prompt:
  model_input = tokenizer(test_data, return_tensors="pt").to("cuda")
  with torch.no_grad():
    output = tokenizer.decode(model.generate(**model_input, max_length=4096,tokenizer=tokenizer, stop_strings='</root>', pad_token_id=2)[0], skip_special_tokens=True)
    print(output)
    model_outputs.append(output)

### Input: ```5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1
6,Grandma's Boysenberry Spread,3,2,12 - 8 oz jars,25.0,120,0,25,0
7,Uncle Bob's Organic Dried Pears,3,7,12 - 1 lb pkgs.,30.0,15,0,10,0
8,Northwoods Cranberry Sauce,3,2,12 - 12 oz jars,40.0,6,0,0,0
```
 ### Output: ''',chef_id''',int''',employee_id''',int''',location_id''',int''',title''',str''',type''',str''',datetime''',datetime''',image''',binary''',text''',text''',employee_id''',int''',constraint ''foreign_key_constraint_name'' foreign key references ''referenced_table_name'' (''referenced_column_name''''',constraint_type''',str''')
  <root>
    <order_id>
      <type>int</type>
      <constraints>primary_key</constraints>
      <constraints>autoincrement</constraints>
      <constraints>not_null</constraints>
    </order_id>
    <product_id>
      <type>int</type>
      <constraints>primary_key</constraints>
      <constraints>autoincrement</constraints>
      <constraints>not_null</constraints>
    </product_id>
   

In [ ]:
pd.DataFrame(model_outputs).to_csv("/content/gdrive/MyDrive/Colab Notebooks/thesis/data/northwind/window_4/test/output/northwind_llama2_recognized.csv")

In [ ]:
new_model = 'llama7b_NorthWind'
trainer.model.save_pretrained(new_model)

In [ ]:
peft_model = PeftModel.from_pretrained(base_model, new_model)
merged_model = peft_model.merge_and_unload()

In [ ]:
output_merged_dir = "/content/gdrive/MyDrive/Colab Notebooks/models/llama7b_northwind"

os.makedirs(output_merged_dir, exist_ok=True)
merged_model.save_pretrained(output_merged_dir, safe_serialization = False)
tokenizer.save_pretrained(output_merged_dir)

Configuration saved in /content/gdrive/MyDrive/Colab Notebooks/models/llama7b_northwind/config.json
Configuration saved in /content/gdrive/MyDrive/Colab Notebooks/models/llama7b_northwind/generation_config.json
The model is bigger than the maximum size per checkpoint (5GB) and is going to be split in 6 checkpoint shards. You can find where each parameters has been saved in the index located at /content/gdrive/MyDrive/Colab Notebooks/models/llama7b_northwind/pytorch_model.bin.index.json.
tokenizer config file saved in /content/gdrive/MyDrive/Colab Notebooks/models/llama7b_northwind/tokenizer_config.json
Special tokens file saved in /content/gdrive/MyDrive/Colab Notebooks/models/llama7b_northwind/special_tokens_map.json


('/content/gdrive/MyDrive/Colab Notebooks/models/llama7b_northwind/tokenizer_config.json',
 '/content/gdrive/MyDrive/Colab Notebooks/models/llama7b_northwind/special_tokens_map.json',
 '/content/gdrive/MyDrive/Colab Notebooks/models/llama7b_northwind/tokenizer.model',
 '/content/gdrive/MyDrive/Colab Notebooks/models/llama7b_northwind/added_tokens.json',
 '/content/gdrive/MyDrive/Colab Notebooks/models/llama7b_northwind/tokenizer.json')